In [1]:
!pip install tensorflow==2.14

In [2]:
!pip install -q -U google-generativeai

In [3]:
!pip install -q google-cloud-aiplatform==1.36.0 google-cloud-discoveryengine==0.11.2 langchain==0.0.327 p

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chainlit 0.7.700 requires click<9.0.0,>=8.1.3, but you have click 7.1.2 which is incompatible.
dask 2023.12.1 requires click>=8.1, but you have click 7.1.2 which is incompatible.
langchain-experimental 0.0.47 requires langchain<0.1,>=0.0.350, but you have langchain 0.0.327 which is incompatible.
pip-tools 7.3.0 requires click>=8, but you have click 7.1.2 which is incompatible.
trulens-eval 0.19.2 requires langchain>=0.0.335, but you have langchain 0.0.327 which is incompatible.


In [4]:
!pip -q install langchain_experimental langchain_core
!pip -q install google-generativeai==0.3.1
!pip -q install google-ai-generativelanguage==0.4.0
!pip -q install langchain-google-genai
!pip -q install trulens_eval

In [5]:
import os
if not os.path.exists('new_articles'):
    os.mkdir('new_articles')

In [6]:
# !wget -q https://www.dropbox.com/s/vs6ocyvpzzncvwh/new_articles.zip
# !unzip -q new_articles.zip -d new_articles

In [7]:
import pathlib
import textwrap
import os
import google.generativeai as genai
import pathlib
import textwrap
from dotenv import load_dotenv
import google.generativeai as genai
from IPython.display import display
from IPython.display import Markdown
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [8]:
load_dotenv()
GOOGLE_API_KEY= os.getenv("GOOGLE_API_KEY")
# GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [9]:
model = genai.GenerativeModel('gemini-pro')

In [10]:
%%time
responce=model.generate_content("What is the meaning of life?")

CPU times: user 16.1 ms, sys: 14.1 ms, total: 30.3 ms
Wall time: 9.12 s


In [11]:
to_markdown(responce.text)
print(responce.text)

The meaning of life is a deep philosophical question that has been pondered by humans for centuries, and there is no single, universally accepted answer. Different people and cultures have different perspectives and beliefs about the meaning of life, and what is meaningful to one person may not be meaningful to another.

Some common themes that emerge in discussions about the meaning of life include:

**Purpose and Fulfillment:** Many people find meaning in their lives through pursuing a purpose or goal that is meaningful to them. This could be a career, a creative endeavor, raising a family, or making a positive impact on the world. When people feel that they are making a difference and contributing to something greater than themselves, they often experience a sense of fulfillment and purpose.

**Relationships and Connections:** Strong relationships with family, friends, and loved ones are often cited as a source of meaning in life. Feeling connected to others and being part of a supp

### RAG(Lanchain+chroma+GeminiAI)

In [12]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [13]:
from langchain.document_loaders import TextLoader
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.prompts import PromptTemplate

# from langchain.schema.output_parser import StrOutputParser

In [14]:
loader = DirectoryLoader('./new_articles/', glob="./*.txt", loader_cls=TextLoader)
documents = loader.load()

In [15]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [16]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [17]:
vectordb = Chroma.from_documents(documents=texts, 
                                 embedding=embeddings)

### RAG template class for True Lense

In [18]:
from trulens_eval import Tru
from trulens_eval.tru_custom_app import instrument
tru = Tru()

ModuleNotFoundError: 
llama-index package is required for instrumenting llama_index apps.
You should be able to install it with pip:

    pip install 'llama-index>=v0.9.14.post3


In [33]:
class RAG_from_scratch:
    @instrument
    def retrieve(self, query: str) -> list:
        """
        Retrieve relevant text from vector store.
        """
        retriever = vectordb.as_retriever()
        results = retriever.get_relevant_documents(query) 
        return results[0]  

    
    @instrument
    def generate_completion(self, query:str, context_str: list) -> str:
        """
        Generate answer from context.
        """
        message = [{
            "role": "user",
            "parts": [
                f"We have provided context information below. \n"
                f"---------------------\n"
                f"{context_str}"
                f"\n---------------------\n"
                f"Given this information, please answer the question: {query}"
            ]
        }]
        model = genai.GenerativeModel('gemini-pro')
        response = model.generate_content(message)
        response_txt = response.text
        return response_txt
        

    @instrument
    def query(self, query:str ) -> str:
        context_str = self.retrieve(query)
        completion = self.generate_completion(query, context_str)
        return completion
    
rag = RAG_from_scratch()


NameError: name 'instrument' is not defined

In [38]:
# # Usage
# rag_instance = RAG_from_scratch()
# my_query = "What is the news about Pando?"
# result = rag_instance.query(my_query)
# print(result)



In [39]:
#Evaluation using TrueLens


In [24]:
# !pip install trulens-eval

In [25]:
!pip install streamlit_javascript


Defaulting to user installation because normal site-packages is not writeable
  Using cached streamlit_javascript-0.1.5-py3-none-any.whl (518 kB)


In [26]:
os.environ["OPENAI_API_KEY"] = "sk-"

In [27]:
from trulens_eval import Feedback, Select
from trulens_eval.feedback import Groundedness
from trulens_eval.feedback.provider.openai import OpenAI as fOpenAI

import numpy as np
import streamlit as st


In [28]:
# Initialize provider class
fopenai = fOpenAI()

grounded = Groundedness(groundedness_provider=fopenai)

# Define a groundedness feedback function
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons, name = "Groundedness")
    .on(Select.RecordCalls.retrieve.rets.collect())
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

# Question/answer relevance between overall question and answer.
f_qa_relevance = (
    Feedback(fopenai.relevance_with_cot_reasons, name = "Answer Relevance")
    .on(Select.RecordCalls.retrieve.args.query)
    .on_output()
)

# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(fopenai.qs_relevance_with_cot_reasons, name = "Context Relevance")
    .on(Select.RecordCalls.retrieve.args.query)
    .on(Select.RecordCalls.retrieve.rets.collect())
    .aggregate(np.mean)
)

✅ In Groundedness, input source will be set to __record__.app.retrieve.rets.collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Answer Relevance, input prompt will be set to __record__.app.retrieve.args.query .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.app.retrieve.args.query .
✅ In Context Relevance, input statement will be set to __record__.app.retrieve.rets.collect() .


In [29]:
from trulens_eval import TruCustomApp
tru_rag = TruCustomApp(rag,
    app_id = 'RAG v1',
    feedbacks = [f_groundedness, f_qa_relevance, f_context_relevance])

In [30]:
with tru_rag as recording:
    rag.query("What is the news about Pando")

In [31]:
tru.get_leaderboard(app_ids=["RAG v1"])

,Answer Relevance,Context Relevance,Groundedness,latency,total_cost
app_id,,,,,
RAG v1,1.0,0.8,0.48125,2.307692,0.0


In [32]:
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Network URL: http://192.168.1.10:8502



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>